
# Group Pipeline — Airline Delay Analysis (Preprocessing & EDA)


**Pipeline order:**
1. **IT24102978 — Handle Missing Data**
2. **IT24102834 — Outlier Removal**
3. **IT24102889 — Encode Categorical Variables**
4. **IT24102942 — Feature Engineering**
5. **IT24102856 — Normalization / Scaling**
6. **IT24102979 — Dimensionality Reduction (PCA)**


In [12]:

# === Setup / Imports ===
import os
import numpy as np
import pandas as pd

# Optional for later steps
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Paths (adjust if needed)
RAW_PATH = "C:\\Users\\Chandupa Weerakkody\\Documents\\AIML_LivePROJECT\\2019.csv"
OUT_DIR = "C:\\Users\\Chandupa Weerakkody\\Documents\\AIML_LivePROJECT\\output"
os.makedirs(OUT_DIR, exist_ok=True)


# === IT24102978 — Handle Missing Data ===
# Minimal, undergrad-friendly

# 1) Load
df = pd.read_csv(RAW_PATH)
print("Shape BEFORE:", df.shape)

# 2) Drop fully empty columns (e.g., 'Unnamed: 20')
df = df.dropna(axis=1, how="all")

# 3) Parse date
if "FL_DATE" in df.columns:
    df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], errors="coerce")

# 4) Known delay-cause NaNs -> 0
cause_cols = ["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]
present_cause = [c for c in cause_cols if c in df.columns]
if present_cause:
    df[present_cause] = df[present_cause].fillna(0)

# 5) Impute remaining numeric/categorical
num_cols = df.select_dtypes(include="number").columns
cat_cols = df.select_dtypes(include="object").columns

# numeric -> median
df[num_cols] = df[num_cols].fillna(df[num_cols].median(numeric_only=True))

# categorical -> mode
for c in cat_cols:
    if df[c].isna().any():
        m = df[c].mode(dropna=True)
        if not m.empty:
            df[c] = df[c].fillna(m.iloc[0])

print("Missing AFTER (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))
print("Shape AFTER missing-data:", df.shape)

# Save intermediate (optional)
df.to_csv(os.path.join(OUT_DIR, "after_missing.csv"), index=False)


# === IT24102834 — Outlier Removal ===
# Remove extreme outliers from DEP_DELAY and ARR_DELAY using IQR*3 (conservative).
# If columns are absent, this section will simply skip.

def remove_outliers_iqr(data, col, k=3.0):
    s = data[col].dropna()
    if s.empty:
        return data
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        return data
    lower, upper = q1 - k*iqr, q3 + k*iqr
    before = len(data)
    data = data[(data[col].isna()) | ((data[col] >= lower) & (data[col] <= upper))].copy()
    removed = before - len(data)
    print(f"{col}: removed {removed} outliers (k={k})")
    return data

for col in ["DEP_DELAY", "ARR_DELAY"]:
    if col in df.columns:
        df = remove_outliers_iqr(df, col, k=3.0)

print("Shape AFTER outlier removal:", df.shape)

# Save intermediate (optional)
df.to_csv(os.path.join(OUT_DIR, "after_outliers.csv"), index=False)


# === IT24102889 — Encode Categorical Variables ===
# Simple: extract date parts; one-hot for CARRIER; top-30 for ORIGIN/DEST then one-hot.

# 1) Date -> numeric
if "FL_DATE" in df.columns:
    df["YEAR"] = df["FL_DATE"].dt.year
    df["MONTH"] = df["FL_DATE"].dt.month
    df["DAY_OF_WEEK"] = df["FL_DATE"].dt.dayofweek  # Mon=0..Sun=6
    # Drop raw date to keep numeric-only downstream
    df = df.drop(columns=["FL_DATE"])

# 2) Limit airport cardinality
for col in ["ORIGIN", "DEST"]:
    if col in df.columns:
        top = df[col].value_counts().head(30).index
        df[col] = df[col].where(df[col].isin(top), "OTHER")

# 3) One-hot encode selected categoricals
cols_to_encode = [c for c in ["OP_UNIQUE_CARRIER", "ORIGIN", "DEST"] if c in df.columns]
df = pd.get_dummies(df, columns=cols_to_encode,
                    prefix=["CARRIER", "ORIGIN", "DEST"],
                    prefix_sep="_", dtype=int)

print("Shape AFTER encoding:", df.shape)

# Save intermediate (optional)
df.to_csv(os.path.join(OUT_DIR, "after_encoding.csv"), index=False)


# === IT24102942 — Feature Engineering ===
# Create simple, interpretable features for later analysis/models.

# Binary delay indicators
if "ARR_DELAY" in df.columns:
    df["IS_DELAYED_ARR"] = (df["ARR_DELAY"] > 0).astype(int)
if "DEP_DELAY" in df.columns:
    df["IS_DELAYED_DEP"] = (df["DEP_DELAY"] > 0).astype(int)

# Weekend flag from DAY_OF_WEEK if present (Mon=0..Sun=6)
if "DAY_OF_WEEK" in df.columns:
    df["IS_WEEKEND"] = df["DAY_OF_WEEK"].isin([5, 6]).astype(int)

# Sum of cause delays if present
cause_cols = ["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]
present_cause = [c for c in cause_cols if c in df.columns]
if present_cause:
    df["TOTAL_DELAY_CAUSES"] = df[present_cause].sum(axis=1)

# Extract hour from CRS_DEP_TIME / CRS_ARR_TIME if they exist (times like HHMM)
def to_hour(val):
    try:
        iv = int(val)
        return iv // 100
    except Exception:
        return np.nan

for col, out_col in [("CRS_DEP_TIME", "CRS_DEP_HOUR"), ("CRS_ARR_TIME", "CRS_ARR_HOUR")]:
    if col in df.columns:
        df[out_col] = df[col].apply(to_hour)

print("Shape AFTER feature engineering:", df.shape)

# Save intermediate (optional)
df.to_csv(os.path.join(OUT_DIR, "after_features.csv"), index=False)


# === IT24102856 — Normalization / Scaling ===
# Standardize numeric features into a separate DataFrame X_scaled (z-scores).
# We keep the original df intact for interpretability.

numeric_cols = df.select_dtypes(include="number").columns.tolist()

# If you want to keep some columns unscaled (e.g., labels), exclude them here:
exclude = []  # e.g., ['ARR_DELAY', 'DEP_DELAY']
scale_cols = [c for c in numeric_cols if c not in exclude]

scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(df[scale_cols].fillna(0)),
    columns=[f"{c}_z" for c in scale_cols],
    index=df.index
)

print("Scaled matrix shape:", X_scaled.shape)

# Save scaled features (optional)
X_scaled.to_csv(os.path.join(OUT_DIR, "scaled_features.csv"), index=False)



# === IT24102979 — Dimensionality Reduction (PCA) ===
# PCA on the standardized features (from previous step). We'll append PCA1/PCA2 to df.

if 'X_scaled' in globals() and not X_scaled.empty:
    pca = PCA(n_components=2, random_state=42)
    comps = pca.fit_transform(X_scaled.values)
    df['PCA1'] = comps[:, 0]
    df['PCA2'] = comps[:, 1]
    print("Explained variance ratio:", pca.explained_variance_ratio_)
else:
    print("PCA skipped: no scaled features.")

print("Final dataframe shape:", df.shape)

# Save final processed dataset with PCA columns included
final_out = os.path.join(OUT_DIR, "final_processed_with_pca.csv")
df.to_csv(final_out, index=False)
print("Saved final dataset ->", final_out)


Shape BEFORE: (7422037, 21)
Missing AFTER (top 10):
FL_DATE              0
OP_UNIQUE_CARRIER    0
SECURITY_DELAY       0
NAS_DELAY            0
WEATHER_DELAY        0
CARRIER_DELAY        0
DISTANCE             0
AIR_TIME             0
ARR_DELAY            0
ARR_TIME             0
dtype: int64
Shape AFTER missing-data: (7422037, 20)
DEP_DELAY: removed 719539 outliers (k=3.0)
ARR_DELAY: removed 46035 outliers (k=3.0)
Shape AFTER outlier removal: (6656463, 20)
Shape AFTER encoding: (6656463, 98)
Shape AFTER feature engineering: (6656463, 102)


MemoryError: Unable to allocate 5.06 GiB for an array with shape (6656463, 102) and data type float64

In [ ]:
# ==============================================================
# Group Pipeline — Airline Delay Analysis (Preprocessing & EDA)
# Memory-safe version (sampling for scaling + PCA)
# Each section is labeled with the member's IT number.
# ==============================================================

import os
import numpy as np
import pandas as pd

# Try to import sklearn; if unavailable, scaling/PCA are skipped.
try:
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    _HAS_SKLEARN = True
except Exception:
    _HAS_SKLEARN = False
    print("Note: scikit-learn not found. Scaling and PCA will be skipped.")

# -----------------------------
# Paths (change if needed)
# -----------------------------
RAW_PATH = "C:\\Users\\Chandupa Weerakkody\\Documents\\AIML_LivePROJECT\\2019.csv"   # portable (recommended)
# RAW_PATH = r"C:\Users\Chandupa Weerakkody\Documents\AIML_LivePROJECT\data\raw\2019.csv"  # absolute option

OUT_DIR = "C:\\Users\\Chandupa Weerakkody\\Documents\\AIML_LivePROJECT\\output"
os.makedirs(OUT_DIR, exist_ok=True)

# Limit rows for scaling/PCA to avoid MemoryError (set None to disable sampling)
MAX_ROWS_FOR_SCALING = 300_000
ONEHOT_PREFIXES = ("CARRIER_", "ORIGIN_", "DEST_")  # columns to exclude from scaling

# ==============================================================
# IT24102978 — Handle Missing Data
# ==============================================================
print("\n=== IT24102978 — Handle Missing Data ===")
df = pd.read_csv(RAW_PATH)
print("Shape BEFORE:", df.shape)

# Drop fully empty columns (e.g., 'Unnamed: 20')
df = df.dropna(axis=1, how="all")

# Parse date if present
if "FL_DATE" in df.columns:
    df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], errors="coerce")

# Known delay-cause NaNs -> 0 (NaN ~= no delay attributed to that cause)
cause_cols = ["CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]
present_cause = [c for c in cause_cols if c in df.columns]
if present_cause:
    df[present_cause] = df[present_cause].fillna(0)

# Impute remaining numeric/categorical
num_cols = df.select_dtypes(include="number").columns
cat_cols = df.select_dtypes(include="object").columns

# numeric -> median
df[num_cols] = df[num_cols].fillna(df[num_cols].median(numeric_only=True))

# categorical -> mode
for c in cat_cols:
    if df[c].isna().any():
        m = df[c].mode(dropna=True)
        if not m.empty:
            df[c] = df[c].fillna(m.iloc[0])

print("Missing AFTER (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))
print("Shape AFTER missing-data:", df.shape)

df.to_csv(os.path.join(OUT_DIR, "after_missing.csv"), index=False)
print("Saved ->", os.path.join(OUT_DIR, "after_missing.csv"))

# ==============================================================
# IT24102834 — Outlier Removal
# ==============================================================
print("\n=== IT24102834 — Outlier Removal (IQR * 3 on DEP_DELAY/ARR_DELAY) ===")

def remove_outliers_iqr(data, col, k=3.0):
    s = data[col].dropna()
    if s.empty:
        return data
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        return data
    lower, upper = q1 - k*iqr, q3 + k*iqr
    before = len(data)
    data = data[(data[col].isna()) | ((data[col] >= lower) & (data[col] <= upper))].copy()
    print(f"{col}: removed {before - len(data)} outliers (k={k})")
    return data

for col in ["DEP_DELAY", "ARR_DELAY"]:
    if col in df.columns:
        df = remove_outliers_iqr(df, col, k=3.0)

print("Shape AFTER outlier removal:", df.shape)
df.to_csv(os.path.join(OUT_DIR, "after_outliers.csv"), index=False)
print("Saved ->", os.path.join(OUT_DIR, "after_outliers.csv"))

# ==============================================================
# IT24102889 — Encode Categorical Variables
# ==============================================================
print("\n=== IT24102889 — Encode Categorical Variables ===")

# Date parts from FL_DATE if present
if "FL_DATE" in df.columns:
    df["YEAR"] = df["FL_DATE"].dt.year
    df["MONTH"] = df["FL_DATE"].dt.month
    df["DAY_OF_WEEK"] = df["FL_DATE"].dt.dayofweek  # Mon=0..Sun=6
    df = df.drop(columns=["FL_DATE"])

# Limit airport cardinality (keep top-30; others -> 'OTHER')
TOP_K_AIRPORTS = 30
for col in ["ORIGIN", "DEST"]:
    if col in df.columns:
        top = df[col].value_counts().head(TOP_K_AIRPORTS).index
        df[col] = df[col].where(df[col].isin(top), "OTHER")

# One-hot encode selected categoricals (simple)
cols_to_encode = [c for c in ["OP_UNIQUE_CARRIER","ORIGIN","DEST"] if c in df.columns]
if cols_to_encode:
    df = pd.get_dummies(df, columns=cols_to_encode, dtype=int)

print("Shape AFTER encoding:", df.shape)
df.to_csv(os.path.join(OUT_DIR, "after_encoding.csv"), index=False)
print("Saved ->", os.path.join(OUT_DIR, "after_encoding.csv"))

# ==============================================================
# IT24102942 — Feature Engineering
# ==============================================================
print("\n=== IT24102942 — Feature Engineering ===")

# Binary delay indicators
if "ARR_DELAY" in df.columns:
    df["IS_DELAYED_ARR"] = (df["ARR_DELAY"] > 0).astype(int)
if "DEP_DELAY" in df.columns:
    df["IS_DELAYED_DEP"] = (df["DEP_DELAY"] > 0).astype(int)

# Weekend flag
if "DAY_OF_WEEK" in df.columns:
    df["IS_WEEKEND"] = df["DAY_OF_WEEK"].isin([5, 6]).astype(int)

# Sum of cause delays if present
present_cause = [c for c in cause_cols if c in df.columns]
if present_cause:
    df["TOTAL_DELAY_CAUSES"] = df[present_cause].sum(axis=1)

# Extract hour from CRS times (HHMM -> hour)
def to_hour(val):
    try:
        iv = int(val)
        return iv // 100
    except Exception:
        return np.nan

for src, out in [("CRS_DEP_TIME","CRS_DEP_HOUR"), ("CRS_ARR_TIME","CRS_ARR_HOUR")]:
    if src in df.columns:
        df[out] = df[src].apply(to_hour)

print("Shape AFTER feature engineering:", df.shape)
df.to_csv(os.path.join(OUT_DIR, "after_features.csv"), index=False)
print("Saved ->", os.path.join(OUT_DIR, "after_features.csv"))

# ==============================================================
# IT24102856 — Normalization / Scaling (MEMORY-SAFE)
# ==============================================================
print("\n=== IT24102856 — Normalization / Scaling (memory-safe) ===")
X_scaled = None
if _HAS_SKLEARN:
    numeric_cols = df.select_dtypes(include="number").columns.tolist()

    # Exclude one-hot columns from scaling (they start with these prefixes)
    scale_cols = [c for c in numeric_cols if not any(c.startswith(p) for p in ONEHOT_PREFIXES)]

    # Choose working rows (sample to fit in memory)
    if (MAX_ROWS_FOR_SCALING is not None) and (len(df) > MAX_ROWS_FOR_SCALING):
        work_idx = df.sample(MAX_ROWS_FOR_SCALING, random_state=42).index
        print(f"Sampling {len(work_idx):,} rows for scaling from {len(df):,}.")
    else:
        work_idx = df.index

    work_X = df.loc[work_idx, scale_cols].fillna(0)

    # Downcast to float32 to reduce memory (sklearn upcasts internally, but this still helps I/O)
    try:
        work_X = work_X.astype("float32")
    except Exception:
        pass

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(work_X.values)
    # Save sample-scaled features (not the full 6.6M rows)
    scaled_sample = pd.DataFrame(X_scaled, columns=[f"{c}_z" for c in scale_cols], index=work_idx)
    scaled_sample.to_csv(os.path.join(OUT_DIR, "scaled_features_SAMPLE.csv"))
    print("Scaled SAMPLE shape:", scaled_sample.shape)
    print("Saved ->", os.path.join(OUT_DIR, "scaled_features_SAMPLE.csv"))
else:
    print("Skipped: scikit-learn not installed.")

# ==============================================================
# IT24102979 — Dimensionality Reduction (PCA) — on scaled SAMPLE
# ==============================================================
print("\n=== IT24102979 — Dimensionality Reduction (PCA on sample) ===")
if _HAS_SKLEARN and (X_scaled is not None):
    pca = PCA(n_components=2, random_state=42)
    comps = pca.fit_transform(X_scaled)  # only on the scaled sample
    pca_df = pd.DataFrame({"PCA1": comps[:, 0], "PCA2": comps[:, 1]}, index=work_idx)

    # Attach PCA1/2 back to the main df (others remain NaN)
    df.loc[work_idx, "PCA1"] = pca_df["PCA1"]
    df.loc[work_idx, "PCA2"] = pca_df["PCA2"]

    # Also save PCA components (sample only)
    pca_df.to_csv(os.path.join(OUT_DIR, "pca_components_SAMPLE.csv"))
    print("PCA sample shape:", pca_df.shape)
    print("Explained variance ratio:", pca.explained_variance_ratio_)
    print("Saved ->", os.path.join(OUT_DIR, "pca_components_SAMPLE.csv"))
else:
    print("Skipped PCA: no scaled sample available or scikit-learn missing.")

print("\nFinal dataframe shape:", df.shape)
final_out = os.path.join(OUT_DIR, "final_processed_with_pca.csv")
df.to_csv(final_out, index=False)
print("Saved final dataset ->", final_out)
